In [ ]:
!pip install evaluate
!pip install rouge_score

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv("/content/outputBartSamsum.csv")
df1 = pd.read_csv("/content/outputPegasusSamsum.csv")
df2 = pd.read_csv("/content/outputProphetnetSamsum.csv")
df3 = pd.read_csv("/content/outputSeasonSamsum.csv")

In [3]:
labelsb = df["Actual Summary"].values
predsb = df["Model Summary"].values
labelsp = df1["Actual Summary"].values
predsp = df1["Model Summary"].values
labelspr = df2["Actual Summary"].values
predspr = df2["Model Summary"].values
labels = df3["Actual Summary"].values
preds = df3["Model Summary"].values


In [4]:
import evaluate
rouge = evaluate.load('rouge')


In [5]:
results = rouge.compute(predictions=predsb,references=labelsb)
print("ROUGE score BART:\n",results)
results = rouge.compute(predictions=predsp,references=labelsp)
print("ROUGE score PEGASUS:\n",results)
results = rouge.compute(predictions=predspr,references=labelspr)
print("ROUGE score ProphetNet:\n",results)
results = rouge.compute(predictions=preds,references=labels)
print("ROUGE score SEASON:\n",results)

ROUGE score BART:
 {'rouge1': 0.4952543887936839, 'rouge2': 0.24644121220272816, 'rougeL': 0.401635826804547, 'rougeLsum': 0.4011246253904617}
ROUGE score PEGASUS:
 {'rouge1': 0.44742091265105066, 'rouge2': 0.2140187908544114, 'rougeL': 0.3493005199247906, 'rougeLsum': 0.3495002205021601}
ROUGE score ProphetNet:
 {'rouge1': 0.4898740926308236, 'rouge2': 0.2440004952638862, 'rougeL': 0.38857379437081985, 'rougeLsum': 0.38819178458974557}
ROUGE score SEASON:
 {'rouge1': 0.5138595938502002, 'rouge2': 0.2648374075439055, 'rougeL': 0.42286743052351583, 'rougeLsum': 0.4227192461243171}


In [6]:
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.translate import meteor
from nltk import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
meteor_val = []
for i in range(len(preds)):
  same_res = round(meteor([word_tokenize(predsb[i])],word_tokenize(labelsb[i])), 4)
  meteor_val.append(same_res)
print("Meteor score BART:",np.mean(meteor_val))

meteor_val = []
for i in range(len(preds)):
  same_res = round(meteor([word_tokenize(predsp[i])],word_tokenize(labelsp[i])), 4)
  meteor_val.append(same_res)
print("Meteor score PEGASUS:",np.mean(meteor_val))

meteor_val = []
for i in range(len(preds)):
  same_res = round(meteor([word_tokenize(predspr[i])],word_tokenize(labelspr[i])), 4)
  meteor_val.append(same_res)
print("Meteor score ProphetNet:",np.mean(meteor_val))

meteor_val = []
for i in range(len(preds)):
  same_res = round(meteor([word_tokenize(preds[i])],word_tokenize(labels[i])), 4)
  meteor_val.append(same_res)
print("Meteor score SEASON:",np.mean(meteor_val))


Meteor score BART: 0.45022688
Meteor score PEGASUS: 0.34517568
Meteor score ProphetNet: 0.39758112
Meteor score SEASON: 0.4782856


In [ ]:
!pip install bert_score
from evaluate import load
bertscore = load("bertscore")

In [10]:
results = bertscore.compute(predictions=predsb, references=labelsb, lang="en")
precision = np.mean(results["precision"])
recall = np.mean(results["recall"])
f1 = np.mean(results["f1"])
print("BERTScore BART->(precision,recall,f1):",precision,recall,f1)

results = bertscore.compute(predictions=predsp, references=labelsp, lang="en")
precision = np.mean(results["precision"])
recall = np.mean(results["recall"])
f1 = np.mean(results["f1"])
print("BERTScore PEGASUS->(precision,recall,f1):",precision,recall,f1)

results = bertscore.compute(predictions=predspr, references=labelspr, lang="en")
precision = np.mean(results["precision"])
recall = np.mean(results["recall"])
f1 = np.mean(results["f1"])
print("BERTScore ProphetNet->(precision,recall,f1):",precision,recall,f1)

results = bertscore.compute(predictions=preds, references=labels, lang="en")
precision = np.mean(results["precision"])
recall = np.mean(results["recall"])
f1 = np.mean(results["f1"])
print("BERTScore SEASON->(precision,recall,f1):",precision,recall,f1)



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore BART->(precision,recall,f1): 0.9155299677848816 0.9184570733070374 0.9168081060409546
BERTScore PEGASUS->(precision,recall,f1): 0.9005578657150268 0.9198963232040406 0.9099476321220398
BERTScore ProphetNet->(precision,recall,f1): 0.9008303033828735 0.9183247685432434 0.9093416741371155
BERTScore SEASON->(precision,recall,f1): 0.9212189059257507 0.9192342805862427 0.9200458987236023


In [ ]:
!pip install moverscore
!pip install pyemd
!pip install pytorch-pretrained-bert

In [4]:
from moverscore import get_idf_dict, word_mover_score
from collections import defaultdict
idf_dict_hyp = get_idf_dict(predsb) # idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = get_idf_dict(labelsb) # idf_dict_ref = defaultdict(lambda: 1.)
scores = word_mover_score(labelsb, predsb, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
mover_score = np.mean(scores)
print("MoverScore BART",mover_score)

idf_dict_hyp = get_idf_dict(predsp) # idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = get_idf_dict(labelsp) # idf_dict_ref = defaultdict(lambda: 1.)
scores = word_mover_score(labelsp, predsp, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
mover_score = np.mean(scores)
print("MoverScore PEGASUS",mover_score)

idf_dict_hyp = get_idf_dict(predspr) # idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = get_idf_dict(labelspr) # idf_dict_ref = defaultdict(lambda: 1.)
scores = word_mover_score(labelspr, predspr, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
mover_score = np.mean(scores)
print("MoverScore ProphetNet",mover_score)

idf_dict_hyp = get_idf_dict(preds) # idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = get_idf_dict(labels) # idf_dict_ref = defaultdict(lambda: 1.)
scores = word_mover_score(labels, preds, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
mover_score = np.mean(scores)
print("MoverScore SEASON",mover_score)

[--------------------------------------------------]
MoverScore BART 0.3218886439835119
MoverScore PEGASUS 0.2684831605352537
MoverScore ProphetNet 0.3084476702286688
MoverScore SEASON 0.3382099184532993


In [17]:
#average length
total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(labels[i]))
avg = total/len(preds)
print("average length of actual summay:",avg)

total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(predsb[i]))
avg = total/len(preds)
print("average length of BART summay:",avg)

total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(predsp[i]))
avg = total/len(preds)
print("average length of PEGASUS summay:",avg)

total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(predspr[i]))
avg = total/len(preds)
print("average length of ProphetNet summay:",avg)

total = 0
for i in range(len(preds)):
  total+= len(word_tokenize(preds[i]))
avg = total/len(preds)
print("average length of SEASON summay:",avg)



average length of actual summay: 23.6944
average length of BART summay: 25.1296
average length of PEGASUS summay: 33.0832
average length of ProphetNet summay: 30.2832
average length of SEASON summay: 22.7792
